### Import related package

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline

### Control memory usage space for GPU

In [2]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.3)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
tf.compat.v1.keras.backend.set_session(sess)

### Preprocessing the data

In [3]:
def read(path):
    return pd.read_csv(path)

In [4]:
def buildTrain(train, pastWeek=4, futureWeek=1, defaultWeek=1):
    X_train, Y_train = [], []
    for i in range(train.shape[0]-futureWeek-pastWeek):
        X = np.array(train.iloc[i:i+defaultWeek])
        X = np.append(X,train["CCSP"].iloc[i+defaultWeek:i+pastWeek])
        X_train.append(X.reshape(X.size))
        Y_train.append(np.array(train.iloc[i+pastWeek:i+pastWeek+futureWeek]["CCSP"]))
    return np.array(X_train), np.array(Y_train)

### Min-max normalization

In [5]:
sc = MinMaxScaler(feature_range = (0, 1))

### Design get_data() to get data

In [6]:
def get_data():
    
    ## Read weekly copper price data
    path = "WeeklyFinalData.csv"
    data = read(path)
    
    date = data["Date"]
    data.drop("Date", axis=1, inplace=True)
    
    ## Add time lag (pastWeek=4, futureWeek=1)
    x_data, y_data = buildTrain(data)
    
#     ## Data split
#     x_train = x_data[0:int(x_data.shape[0]*0.8)]
#     x_test = x_data[int(x_data.shape[0]*0.8):]
    
#     y_train = y_data[0:int(y_data.shape[0]*0.8)]
#     y_test = y_data[int(y_data.shape[0]*0.8):]
    
    ## Normalize
    x_train_scaled = sc.fit_transform(x_data)
    y_train_scaled = sc.fit_transform(y_data)
    
    
    return (x_train_scaled, y_train_scaled)

### Network class

In [18]:
class Network():
    
    def __init__(self, nb_neuro):
        
        x_train_scaled, y_train_scaled = get_data()
        
        # Stop criteria - threshold
        self.threshold_for_error = 1e-1*5
        self.threshold_for_lr = 1e-4
        
        # Input data
        self.x = tf.convert_to_tensor(x_train_scaled, np.float32)
        self.y = tf.convert_to_tensor(y_train_scaled, np.float32)
        
        # Learning rate
        self.learning_rate = 1e-2
        
        # Optimizer
#         self.optimizer = tf.optimizers.SGD(self.learning_rate)
        
         # Hidden layer I
        self.n_neurons_in_h1 = nb_neuro
        self.W1 = tf.Variable(tf.random.truncated_normal([self.x.shape[1], self.n_neurons_in_h1], mean=0, stddev=1))
        self.b1 = tf.Variable(tf.random.truncated_normal([self.n_neurons_in_h1], mean=0, stddev=1))

        # Output layer
        self.Wo = tf.Variable(tf.random.truncated_normal([self.n_neurons_in_h1, self.y.shape[1]], mean=0, stddev=1))
        self.bo = tf.Variable(tf.random.truncated_normal([self.y.shape[1]], mean=0, stddev=1))

        # Whether the network is acceptable
        self.acceptable = False
        
        # forward operation
    def forward(self,  reg_strength= 0):
        with tf.GradientTape() as tape:

            y1 = tf.nn.relu((tf.matmul(self.x, self.W1)+self.b1))
            yo = (tf.matmul(y1,self.Wo)+self.bo)

            # performance measure
            diff = yo-self.y
            loss = tf.reduce_mean(diff**2) + reg_strength * (tf.nn.l2_loss(self.W1) + tf.nn.l2_loss(self.Wo) + tf.nn.l2_loss(self.b1) + tf.nn.l2_loss(self.bo))*2

        return(yo, loss, tape)

    # backward operation
    def backward_SGD(self,tape,loss):

        optimizer = tf.optimizers.SGD(self.learning_rate)
        gradients = tape.gradient(loss, [self.W1, self.Wo, self.b1, self.bo])
        optimizer.apply_gradients(zip(gradients, [self.W1, self.Wo, self.b1, self.bo]))
        
    def backward_RMS(self,tape,loss):

        optimizer = tf.keras.optimizers.RMSprop(self.learning_rate)
        gradients = tape.gradient(loss, [self.W1, self.Wo, self.b1, self.bo])
        optimizer.apply_gradients(zip(gradients, [self.W1, self.Wo, self.b1, self.bo]))

### Matching module

In [19]:
# tunning the parameter
def matching_module(network):

    
    while True:

        yo, loss, tape = network.forward()

        if tf.reduce_all(tf.math.abs(yo-network.y) <= network.threshold_for_error):
            network.acceptable = True
            return("Acceptable")


        else:
                # Save the current papameter
                W1_pre = network.W1
                Wo_pre = network.Wo
                b1_pre = network.b1
                bo_pre = network.bo
                loss_pre = loss

                # tuning and check the loss performance of the next step
                network.backward_SGD(tape,loss)
                yo, loss, tape = network.forward()

                # Confirm whether the adjusted loss value is smaller than the current one
                if loss < loss_pre:

                    # Multiply the learning rate by 1.2
                    network.learning_rate *= 1.2

                # On the contrary, reduce the learning rate
                else:

                    # Identify whether the current learning rate is less than the threshold
                    if network.learning_rate < network.threshold_for_lr:
                        network.acceptable = False
                        # If true, return the current model parameters
                        return("Unacceptable")

                    # On the contrary, maintain the original parameter and adjust the learning rate
                    else:
                        network.W1 = W1_pre
                        network.Wo = Wo_pre
                        network.b1 = b1_pre
                        network.bo = bo_pre
                        network.learning_rate *= 0.7

### Regularizing module

In [20]:
def regularization(network):

    if network.acceptable:

        W1_pre, b1_pre, Wo_pre, bo_pre = network.W1, network.b1, network.Wo, network.bo
#         network.optimizer = tf.keras.optimizers.RMSprop(network.learning_rate)
        yo, loss, tape = network.forward(1e-2)
    
        for _ in range(100):
            
            loss_pre = loss
            network.backward_RMS(tape, loss)
            yo, loss, tape = network.forward(1e-2)
           
            if loss <= loss_pre:
                if tf.reduce_all(tf.math.abs(yo-network.y) <= network.threshold_for_error):
                    network.learning_rate *= 1.2

                else:
                    network.W1, network.b1, network.Wo, network.bo = W1_pre, b1_pre, Wo_pre, bo_pre
                    return("Acceptable SLFN")
#                     break

            else:

                network.W1, network.b1, network.Wo, network.bo = W1_pre, b1_pre, Wo_pre, bo_pre

                if network.learning_rate > network.threshold_for_lr:
                    network.learning_rate *= 0.7

                else:
                    return("Acceptable SLFN")
#                     break

    else:
        return("The input network should be an acceptable network.")

### Reorganizing module

In [21]:
def reorganizing(network):
    
    if network.acceptable:
        
        k = 1
        p = network.n_neurons_in_h1

        while True:
            if k > p:
                return("Finished!")

            else:
                regularization(network)
                network_pre = network
                network.acceptable = False
                network.W1 = tf.Variable(tf.concat([network.W1[:,:k-1],network.W1[:,k:]],1))
                network.b1 = tf.Variable(tf.concat([network.b1[:k-1],network.b1[k:]],0))
                network.Wo = tf.Variable(tf.concat([network.Wo[:k-1,:],network.Wo[k:,:]],0))

    #             print(network.W1.shape, network.Wo.shape, network.b1.shape)
                matching_module(network)

                if network.acceptable:
                    print("A.neuro index: %d, nb of neuro:%d" %(k, p))
                    p-=1

                else:
                    network = network_pre
                    print("U.neuro index: %d, nb of neuro:%d" %(k, p))
                    k+=1
    else:
        return ("The input network should be an acceptable network.")

### Cramming module

In [22]:
def cramming_module(network):
    ## Set the random seed
    tf.random.set_seed(5)
    
    ## Find unsatisfied situation
    yo, loss, tape = network.forward()
    
    ## Unsatisfied situation
    undesired_index = tf.where(tf.math.abs(yo-network.y) > network.threshold_for_error)
    
    if undesired_index.shape[0]==1:
        
        undesired_index = undesired_index[0][0]
        undesired_data = tf.reshape(network.x[undesired_index],[1,-1])

        ## Remove the only data that does not meet the error term
        left_data = network.x[:undesired_index,:]
        right_data = network.x[undesired_index+1:,:]
        remain_tensor = tf.concat([left_data, right_data], 0)

        while True:

            ## Find m-vector r
            gamma = tf.random.uniform(shape=[1,network.x.shape[1]])

            subtract_undesired_data = tf.subtract(remain_tensor, gamma)
            matmul_value = tf.matmul(gamma,tf.transpose(subtract_undesired_data))

            ## Find the gamma

            while True:
               
                ## Find the tiny value: zeta
                zeta = tf.random.uniform(shape=[1])

                if np.all(tf.less(tf.multiply(tf.add(zeta,matmul_value),tf.subtract(zeta,matmul_value)),0)):
                    break

            break

        ## The weight of input layer to hidden layer I
        w10 = gamma
        w11 = gamma
        w12 = gamma

        W1_new = tf.transpose(tf.concat([w10,w11,w12],0))

        ## The bias of input layer to hidden layer I
        matual_value = tf.matmul(gamma,tf.transpose(undesired_data))

        b10 = tf.subtract(zeta,matual_value)
        b11 = -matual_value
        b12 = tf.subtract(-1*zeta,matual_value)

        b1_new = tf.reshape(tf.concat([b10,b11,b12],0),[3])

        ## The weight of hidden layer I to output layer
        gap = network.y[undesired_index]-yo[undesired_index]

        wo0 = gap/zeta
        wo1 = (-2*gap)/zeta
        wo2 = gap/zeta

        Wo_new = tf.reshape(tf.concat([wo0,wo1,wo2],0),[-1,1])

        ## Add new neuroes to the network
        network.W1 = tf.concat([network.W1, W1_new],1)
        network.b1 = tf.concat([network.b1, b1_new],0)
        network.Wo = tf.concat([network.Wo, Wo_new],0)
        
        if tf.reduce_all(tf.math.abs(yo-network.y) <= network.threshold_for_error):
            network.acceptable = True
            print("Cramming finished!")
            
    else:
        print("Undesired data > 1")

### Construct a instance of network
- trained through the matching module, reorganizing module, and cramming module

In [23]:
def main():
    
    network = Network(32)
    matching_module(network)
    print(network.acceptable)
    
    if not network.acceptable:
        cramming_module(network)
        
    reorganizing(network)

In [24]:
if __name__ == "__main__":
    main()

True
A.neuro index: 1, nb of neuro:32
A.neuro index: 1, nb of neuro:31
A.neuro index: 1, nb of neuro:30
A.neuro index: 1, nb of neuro:29
A.neuro index: 1, nb of neuro:28
A.neuro index: 1, nb of neuro:27
A.neuro index: 1, nb of neuro:26
A.neuro index: 1, nb of neuro:25
A.neuro index: 1, nb of neuro:24
A.neuro index: 1, nb of neuro:23
U.neuro index: 1, nb of neuro:22
U.neuro index: 2, nb of neuro:22
U.neuro index: 3, nb of neuro:22
U.neuro index: 4, nb of neuro:22
U.neuro index: 5, nb of neuro:22
U.neuro index: 6, nb of neuro:22
U.neuro index: 7, nb of neuro:22
U.neuro index: 8, nb of neuro:22
U.neuro index: 9, nb of neuro:22
U.neuro index: 10, nb of neuro:22
U.neuro index: 11, nb of neuro:22
U.neuro index: 12, nb of neuro:22
U.neuro index: 13, nb of neuro:22
U.neuro index: 14, nb of neuro:22
U.neuro index: 15, nb of neuro:22
U.neuro index: 16, nb of neuro:22
U.neuro index: 17, nb of neuro:22
U.neuro index: 18, nb of neuro:22
U.neuro index: 19, nb of neuro:22
U.neuro index: 20, nb of neu